In [1]:
%load_ext autoreload
%autoreload 2

from utils import *
import pyclesperanto as cle
import numpy as np
import napari
from magicgui import magicgui, magic_factory
import pandas as pd
cle.select_device("RTX")

(OpenCL) NVIDIA GeForce RTX 4070 SUPER (OpenCL 3.0 CUDA)
	Vendor:                      NVIDIA Corporation
	Driver Version:              560.70
	Device Type:                 GPU
	Compute Units:               56
	Global Memory Size:          12281 MB
	Maximum Object Size:         3070 MB
	Max Clock Frequency:         2505 MHz
	Image Support:               Yes

In [2]:
path = "../../04_uCT/AD51/"
scan = load_scan(path, logging=True)
# scan.reslice()

Loading images from:  ../../04_uCT/AD51/Slices/
Loaded stack with shape (1780, 322, 2267) and a size of 2.60 GB in 1.79 s.


In [3]:
subscan = scan[200:230]

In [4]:
@magicgui(auto_call=True)
def segment(scan: ImageData,
            logging: bool = True,
            otsu_sigma: float = 0.6,
            particle_enlarge_radius: int = 1,
            particle_mask_sigma: float = 0.1,
            particle_erosions: int = 0,
            smooth_labels_radius: int = 2) -> LabelsData:
    mask = segment_scan(scan, logging, otsu_sigma, particle_enlarge_radius, particle_mask_sigma, particle_erosions, smooth_labels_radius)
    return mask


def process_subset_in_napari(scan: ImageData, subset_size: int = 30):
    @magicgui(auto_call=True)
    def segment(scan: ImageData,
                logging: bool = True,
                otsu_sigma: float = 0.6,
                particle_enlarge_radius: int = 1,
                particle_mask_sigma: float = 0.1,
                particle_erosions: int = 0,
                smooth_labels_radius: int = 2) -> LabelsData:
        mask = segment_scan(scan, logging, otsu_sigma, particle_enlarge_radius, particle_mask_sigma, particle_erosions, smooth_labels_radius)
        return mask
    
    n_slices, _, _ = scan.shape
    subscan = scan[n_slices//2-subset_size//2:n_slices//2+subset_size//2]
    
    viewer = napari.Viewer()
    viewer.add_image(subscan)
    viewer.window.add_dock_widget(segment, area='right')
    

In [5]:
mask = segment(subscan)

Smoothed image.
Finished otsu masks.
Finished refined particle mask.
Created full segmentation.


In [6]:
viewer = napari.Viewer()
viewer.add_image(subscan)
viewer.add_labels(mask)

<Labels layer 'mask' at 0x254bc1079d0>

In [5]:
viewer = napari.Viewer()
viewer.add_image(subscan)
viewer.window.add_dock_widget(segment, area='right')

Smoothed image.
Finished otsu masks.
Finished refined particle mask.
Created full segmentation.
